In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from matplotlib.lines import Line2D
from sklearn.metrics import mean_absolute_percentage_error as mean_ape
from sklearn.metrics import root_mean_squared_error as root_mse

sns.set(style="whitegrid")

In [ ]:
model_data_no_journal = pd.read_parquet("data/model_data_no_journal.parquet")
model_data_no_journal = model_data_no_journal[model_data_no_journal['alignement_politique'] != 'autre']
model_data_no_journal['pres_votes_share'] = model_data_no_journal['pres_dummy'] * model_data_no_journal['pres_votes_share']

model_data = pd.read_parquet("data/model_data.parquet")
model_data = model_data[model_data['alignement_politique'] != 'autre']
model_data['pres_votes_share'] = model_data['pres_dummy'] * model_data['pres_votes_share']

In [ ]:
outcome = "quotes_share"

Approche économétrique négative : on étudie l'écart des comportements réels à des comportements théoriques.
En l'occurrence à des normes encadrant les pratiques journalistes.
La méthode ici consiste à modéliser ces normes, puis à évaluer dans quelle mesure elles permettent de prédire les comportements réels, et à observer comment ces derniers s'en écartent. 
On s'intéresse aux R2 et aux résidus.

Dans l'idéal, les normes doivent être parfaitement connues.
Exemple de la règle des tiers, les articles se répartissent au tiers entre le gouvernement, la majorité et l'opposition.
Mais d'application délicate car elle concerne des unités fluctuant au cours du temps (les partis qui gouvernent, les partis qui soutiennent le gouvernement sans y participer, les partis qui s'opposent au gouvernement).
Et dépourvue d'utilité pour nous : ne permet pas d'étudier des outcomes spécifiques à l'extrême-droite, qui sera toujours diluée dans l'opposition.

Modèle alternatif : les journaux accordent une fraction $\alpha$ des articles à la nuance politique qui gouverne, et répartissent les articles restants entre les nuances politiques en fonction de leur poids électoral. Ce qu'on peut écrire, en notant $G_i$ la participation au gouvernement et $T_i$ les résultats à la dernière élection législative : 
$$Y_{i} = \alpha + \beta G_i + \gamma T_i$$

La difficulté est la valeur de $\alpha$ n'est pas connue *a priori* (pas d'argument dans la littérature pour étayer $\alpha = \frac{1}{3}$ par exemple). 
Il faut l'estimer à partir des données.
Risque que le raisonnement deviennent circulaire : si l'on tente de mesurer l'écart des comportements réels à une norme en définissant celle-ci à partir de ces mêmes comportements, on se retrouve en fait à commenter la performance prédictive du modèle.
Pour minimiser ce risque, il faut que le nombre de paramètres estimés soit le plus faible possible (comme dans le modèle précédent, supposant que la norme est identique pour les nuances politiques, les journaux et les périodes).

*NB : une difficulté supplémentaire pour étudier le phénomène de légitimation de l'extrême droite : des évolutions surviennent au cours du temps, soit que les normes évoluent, soit que les écarts aux normes s'amplifient. Estimer les paramètres de la norme sur toute la période conduit à sous-estimer la transformation des comportements réels. Dans ces circonstances, il vaut mieux :*
- *Estimer sur une période de référence, et appliquer la norme ainsi obtenue aux autres périodes.*
- *Estimer plusieurs normes sur plusieurs périodes, mais cela est une manière de revenir à une approche économétrique traditionnelle.*
*D'un autre côté, dans le modèle actuel, aucun paramètre n'est spécifique à l'extrême droite.*

# Estimation de la norme

In [ ]:
# Attention ci-dessous les modèles estimés n'a pas 1 mais 2 paramètres !

In [ ]:
X = model_data_no_journal[["na_share", "government", "pres_votes_share", "leg_votes_share"]]
y = model_data_no_journal[outcome]
model = sm.OLS(y, X).fit(cov_type='HC3')

y_pred = model.predict(X)
mape = mean_ape(y, y_pred)
rmspe = root_mse(y, y_pred) / y.mean()

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"R2: {100*model.rsquared:.2f}%")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

In [ ]:
cutoff = pd.Period('2012-06', freq='M')
X = model_data_no_journal[model_data_no_journal["month"] <= cutoff][["na_share", "government", "pres_votes_share", "leg_votes_share"]]
y = model_data_no_journal[model_data_no_journal["month"] <= cutoff][outcome]
model = sm.OLS(y, X).fit(cov_type='HC3')

y_pred = model.predict(X)
mape = mean_ape(y, y_pred)
rmspe = root_mse(y, y_pred) / y.mean()

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"R2: {100*model.rsquared:.2f}%")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

# Ecarts

## Selon les partis politiques

In [ ]:
model_data_no_journal['y_pred'] = model.predict(model_data_no_journal[["na_share", "government", "pres_votes_share", "leg_votes_share"]])
model_data_no_journal['rel_residuals'] = 100 * (model_data_no_journal['y_pred'] - model_data_no_journal[outcome]) / model_data_no_journal[outcome]

In [ ]:
summary = []

for alignment in model_data_no_journal['alignement_politique'].unique():
    subset_data = model_data_no_journal[model_data_no_journal['alignement_politique'] == alignment]
    y = subset_data[outcome]
    y_pred = subset_data['y_pred']
    mape = mean_ape(y, y_pred)
    rmspe = root_mse(y, y_pred) / y.mean()
    summary.append({
        'nuance politique': alignment,
        'MAPE': mape,
        'RMSPE': rmspe
    })

pd.DataFrame(summary).style.hide(axis=0)

In [ ]:
plot_data = model_data_no_journal.copy()
plot_data['month'] = plot_data['month'].dt.to_timestamp()

alignment_groups = [
    (['extremegauche_gaucheradicale', 'extremedroite_droiteradicale'],
     {'extremegauche_gaucheradicale': 'crimson',
      'extremedroite_droiteradicale': 'royalblue'}),
    (['centredroite_droite', 'centregauche_gauche', 'centre'],
     {'centredroite_droite': 'cornflowerblue',
      'centregauche_gauche': 'orchid',
      'centre': 'goldenrod'})]

In [ ]:
alignment = 'extremedroite_droiteradicale'
subset_data = plot_data[plot_data['alignement_politique'] == alignment].copy()
subset_data['MA_observed'] = subset_data[outcome].rolling(window=6).mean()
subset_data['MA_rel'] = subset_data['rel_residuals'].rolling(window=6).mean()

fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

sns.lineplot(data=subset_data, x='month', y=outcome, ax=axes[0], label='Valeurs observées', alpha=0.2, color='crimson', linestyle='-')
sns.lineplot(data=subset_data, x='month', y='MA_observed', ax=axes[0], label='Moyenne mobile', alpha=0.65, color='crimson', linestyle='dashdot')
sns.lineplot(data=subset_data, x='month', y='y_pred', ax=axes[0], label='Valeurs prédites', alpha=1, color='teal', linestyle='dotted')
axes[0].set_title("Valeurs réelles et prédites")
axes[0].set_ylabel('')

sns.lineplot(data=subset_data, x='month', y='rel_residuals', ax=axes[1], label='Résidus relatifs', alpha=0.3, color='crimson', linestyle='-')
sns.lineplot(data=subset_data, x='month', y='MA_rel', ax=axes[1], label='Moyenne mobile', color='crimson', linestyle='dashdot')
sns.lineplot(data=subset_data, x='month', y='y_pred', ax=axes[1], label='Valeurs prédites', alpha=1, color='teal', linestyle='dotted')
axes[1].set_title("Résidus relatifs (%)")
axes[1].set_xlabel('')
axes[1].set_ylabel('')

plt.suptitle("Proportion des citations de personnalités d'extrême droite")
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 12), sharex=True)

for ax, (political_alignments, colors) in zip(axes, alignment_groups):
    alignment_handles = []

    for alignment in political_alignments:
        subset_data = plot_data[plot_data['alignement_politique'] == alignment].copy()
        subset_data['MA'] = subset_data[outcome].rolling(window=6).mean()
        
        ax.plot(subset_data['month'], subset_data[outcome], label=None,
                alpha=0.2, color=colors[alignment], linestyle='-')
        ax.plot(subset_data['month'], subset_data['MA'], label=None,
                alpha=0.65, color=colors[alignment], linestyle='dashdot')
        ax.plot(subset_data['month'], subset_data['y_pred'], label=None,
                alpha=1, color=colors[alignment], linestyle='dotted')
        
        alignment_handles.append(Line2D([0], [0], color=colors[alignment], lw=2, label=alignment))

    alignment_legend = ax.legend(handles=alignment_handles, title="Nuance politique", loc="upper left")
    ax.add_artist(alignment_legend)

    line_type_handles = [
        Line2D([0], [0], color='black', lw=2, linestyle='-', label="Moyenne mensuelle"),
        Line2D([0], [0], color='black', lw=2, linestyle='dashdot', label="Moyenne mobile sur 6 mois"),
        Line2D([0], [0], color='black', lw=2, linestyle='dotted', label="Prédictions")]
    ax.legend(handles=line_type_handles, title="Valeurs", loc="upper right")

axes[-1].set_xlabel("")

plt.suptitle("Répartition des citations entre les nuances politiques\nValeurs observées et prédites")
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 12), sharex=True)

for ax, (political_alignments, colors) in zip(axes, alignment_groups):
    alignment_handles = []

    for alignment in political_alignments:
        subset_data = plot_data[plot_data['alignement_politique'] == alignment].copy()
        subset_data['MA'] = subset_data['rel_residuals'].rolling(window=6).mean()
        
        ax.plot(subset_data['month'], subset_data['rel_residuals'], label=None,
                alpha=0.2, color=colors[alignment], linestyle='-')
        ax.plot(subset_data['month'], subset_data['MA'], label=None,
                alpha=0.65, color=colors[alignment], linestyle='dashdot')

        alignment_handles.append(Line2D([0], [0], color=colors[alignment], lw=2, label=alignment))

    alignment_legend = ax.legend(handles=alignment_handles, title="Nuance politique", loc="upper left")
    ax.add_artist(alignment_legend)

    line_type_handles = [
        Line2D([0], [0], color='black', lw=2, linestyle='-', label="Moyenne mensuelle"),
        Line2D([0], [0], color='black', lw=2, linestyle='dashdot', label="Moyenne mobile sur 6 mois")
    ]
    ax.legend(handles=line_type_handles, title="Valeurs", loc="upper right")

axes[-1].set_xlabel("")

plt.suptitle("Répartition des citations entre les nuances politiques\nRésidus relatifs (%)")
plt.tight_layout()
plt.show()

## Selon les journaux

In [ ]:
model_data['y_pred'] = model.predict(model_data[["na_share", "government", "pres_votes_share", "leg_votes_share"]])
model_data['rel_residuals'] = 100 * (model_data['y_pred'] - model_data[outcome]) / model_data[outcome]

In [ ]:
summary = []

for journal in model_data['journal'].unique():
    subset_data = model_data[model_data['journal'] == journal]
    y = subset_data[outcome]
    y_pred = subset_data['y_pred']
    mape = mean_ape(y, y_pred)
    rmspe = root_mse(y, y_pred) / y.mean()
    summary.append({
        'journal': journal,
        'MAPE': mape,
        'RMSPE': rmspe
    })

pd.DataFrame(summary).style.hide(axis=0)

In [ ]:
plot_data = model_data.copy()
plot_data['month'] = plot_data['month'].dt.to_timestamp()

colors = {
    'Le Figaro': 'goldenrod',
    'Libération': 'limegreen',
    'Le Monde': 'orchid',
    'La Croix': 'skyblue',
    'Mediapart': 'crimson'}

alignments = [
    "extremedroite_droiteradicale",
    "centredroite_droite",
    "centre",
    "centregauche_gauche",
    "extremegauche_gaucheradicale"]
n_alignments = len(alignments)

In [ ]:
fig, axes = plt.subplots(n_alignments, 1, figsize=(16, 4 * n_alignments), sharex=True)

for i, alignment in enumerate(alignments):
    ax = axes[i]
    subset_data = plot_data[plot_data['alignement_politique'] == alignment]

    for journal in subset_data['journal'].unique():
        if journal == 'Mediapart': continue
        sub_subset_data = subset_data[subset_data['journal'] == journal].copy()
        sub_subset_data['MA'] = sub_subset_data[outcome].rolling(window=12).mean()
        ax.plot(sub_subset_data['month'], sub_subset_data[outcome], label=journal,
                color=colors[journal], linestyle='-')

    ax.plot(subset_data['month'], subset_data['y_pred'], color='black', alpha=0.8, linestyle='dotted')
    ax.set_title(f"{alignment}")
    ax.legend()

plt.suptitle("""
Proportion des citations de personnalités de plusieurs nuances politiques dans 5 journaux\n
Valeurs réelles et prédites - moyennes mobiles sur 12 mois
""")
plt.tight_layout()
plt.show()

In [ ]:
alignments = [
    "extremedroite_droiteradicale",
    "centredroite_droite",
    "centre",
    "centregauche_gauche",
    "extremegauche_gaucheradicale"
]
n_alignments = len(alignments)

fig, axes = plt.subplots(n_alignments, 1, figsize=(16, 4 * n_alignments), sharex=True)

for i, alignment in enumerate(alignments):
    ax = axes[i]
    subset_data = plot_data[plot_data['alignement_politique'] == alignment]

    for journal in subset_data['journal'].unique():
        sub_subset_data = subset_data[subset_data['journal'] == journal].copy()
        sub_subset_data['MA'] = sub_subset_data['rel_residuals'].rolling(window=12).mean()
        ax.plot(sub_subset_data['month'], sub_subset_data['MA'], label=journal,
                color=colors[journal], linestyle='-')

    ax.plot(subset_data['month'], subset_data['y_pred'], color='black', alpha=0.8, linestyle='dotted')
    ax.set_title(f"{alignment}")
    ax.legend()

plt.suptitle("""
Proportion des citations de personnalités de plusieurs nuances politiques dans 5 journaux\n
Résidus relatifs (%) - moyennes mobiles sur 12 mois
""")
plt.tight_layout()
plt.show()